<a href="https://colab.research.google.com/github/mmarin11/Colabfiles/blob/main/Practica6/Practica6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import argparse
import numpy as np
import pandas as pd


def cargar_datos(path_csv: str) -> pd.DataFrame:
    df = pd.read_csv(path_csv)
    # Normalizar nombres de columnas usadas
    df = df.rename(columns={
        'Siblings/Spouses Aboard': 'SibSp',
        'Parents/Children Aboard': 'Parch'
    })
    # Validaciones básicas
    req = {'Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch'}
    faltan = req - set(df.columns)
    if faltan:
        raise ValueError(f"Faltan columnas requeridas en el CSV: {faltan}")
    return df


def ejercicio_1(df: pd.DataFrame):
    """Proporción de supervivencia por combinación de Sex y Pclass; mayor y menor tasa."""
    rates = df.groupby(['Sex', 'Pclass'])['Survived'].mean().sort_values(ascending=False)
    highest_idx = rates.idxmax()
    lowest_idx = rates.idxmin()
    highest = (highest_idx, float(rates.loc[highest_idx]))
    lowest = (lowest_idx, float(rates.loc[lowest_idx]))
    return rates, highest, lowest


def ejercicio_2(df: pd.DataFrame):
    """Familias grandes (FamilySize=SibSp+Parch > 3) y su proporción de supervivencia."""
    df = df.copy()
    df['FamilySize'] = df['SibSp'] + df['Parch']
    grandes = df[df['FamilySize'] > 3]
    count_grandes = int(len(grandes))
    survival_grandes = float(grandes['Survived'].mean()) if count_grandes > 0 else np.nan
    return df, count_grandes, survival_grandes


def ejercicio_3(df: pd.DataFrame):
    """Segmentación por grupos de edad (Menor de Edad <18, Mayor de Edad >=18)."""
    def age_cat(age):
        if pd.isna(age):
            return np.nan
        return 'Menor de Edad' if age < 18 else 'Mayor de Edad'
    df = df.copy()
    df['AgeGroup'] = df['Age'].apply(age_cat)
    return df


def ejercicio_4(df: pd.DataFrame):
    """Promedios con NumPy vs Pandas (ignorando nulos)."""
    mean_age_np = float(np.nanmean(df['Age'].to_numpy()))
    mean_fare_np = float(np.nanmean(df['Fare'].to_numpy()))
    mean_age_pd = float(df['Age'].mean())
    mean_fare_pd = float(df['Fare'].mean())
    consistentes = np.isclose(mean_age_np, mean_age_pd) and np.isclose(mean_fare_np, mean_fare_pd)
    return (mean_age_np, mean_fare_np, mean_age_pd, mean_fare_pd, bool(consistentes))


def ejercicio_5(df: pd.DataFrame, bins: int = 5):
    """
    Intervalos equidistantes en Fare con numpy.linspace y análisis de
    número de pasajeros y proporción de supervivencia por intervalo.
    """
    fare_min, fare_max = df['Fare'].min(), df['Fare'].max()
    edges = np.linspace(fare_min, fare_max, bins + 1)  # p.ej. 5 intervalos = 6 bordes
    df = df.copy()
    df['FareInterval'] = pd.cut(df['Fare'], bins=edges, include_lowest=True)
    conteo = df['FareInterval'].value_counts().sort_index()
    survive_by_bin = df.groupby('FareInterval')['Survived'].mean()
    return df, edges, conteo, survive_by_bin


def main():

    csv_path = "drive/MyDrive/ClassFiles/titanic.csv"


    df = cargar_datos(csv_path)

    # ---- Ejercicio 1
    rates, highest, lowest = ejercicio_1(df)
    print("\n=== Ejercicio 1: Supervivencia por Sex x Pclass ===")
    print(rates.to_frame(name='SurvivalRate').to_string())
    print(f"\nTasa más alta : {highest[0]} -> {highest[1]:.3f}")
    print(f"Tasa más baja : {lowest[0]}  -> {lowest[1]:.3f}")

    # ---- Ejercicio 2
    df2, n_grandes, surv_grandes = ejercicio_2(df)
    print("\n=== Ejercicio 2: Familias grandes (FamilySize > 3) ===")
    print(f"Número de pasajeros en familias grandes: {n_grandes}")
    print(f"Proporción de supervivencia en familias grandes: {surv_grandes:.3f}" if not np.isnan(surv_grandes)
          else "Proporción de supervivencia en familias grandes: N/A (no hay casos)")

    # ---- Ejercicio 3
    df3 = ejercicio_3(df2)
    print("\n=== Ejercicio 3: Segmentación por grupos de edad ===")
    print(df3['AgeGroup'].value_counts(dropna=False).to_string())

    # ---- Ejercicio 4
    mean_age_np, mean_fare_np, mean_age_pd, mean_fare_pd, ok = ejercicio_4(df3)
    print("\n=== Ejercicio 4: Promedios NumPy vs Pandas ===")
    print(f"Edad media -> NumPy: {mean_age_np:.3f} | Pandas: {mean_age_pd:.3f}")
    print(f"Fare medio -> NumPy: {mean_fare_np:.3f} | Pandas: {mean_fare_pd:.3f}")
    print("¿Consistentes? ->", "Sí" if ok else "No")

    # ---- Ejercicio 5
    df5, edges, conteo, surv_by_bin = ejercicio_5(df3, bins=5)
    print("\n=== Ejercicio 5: Intervalos de Fare y supervivencia por intervalo ===")
    print("Bordes de intervalos (linspace):", [round(e, 4) for e in edges.tolist()])
    print("\nPasajeros por intervalo:")
    print(conteo.to_string())
    print("\nSupervivencia por intervalo:")
    print(surv_by_bin.to_string())


if __name__ == "__main__":
    main()


=== Ejercicio 1: Supervivencia por Sex x Pclass ===
               SurvivalRate
Sex    Pclass              
female 1           0.968085
       2           0.921053
       3           0.500000
male   1           0.368852
       2           0.157407
       3           0.137026

Tasa más alta : ('female', np.int64(1)) -> 0.968
Tasa más baja : ('male', np.int64(3))  -> 0.137

=== Ejercicio 2: Familias grandes (FamilySize > 3) ===
Número de pasajeros en familias grandes: 62
Proporción de supervivencia en familias grandes: 0.161

=== Ejercicio 3: Segmentación por grupos de edad ===
AgeGroup
Mayor de Edad    757
Menor de Edad    130

=== Ejercicio 4: Promedios NumPy vs Pandas ===
Edad media -> NumPy: 29.471 | Pandas: 29.471
Fare medio -> NumPy: 32.305 | Pandas: 32.305
¿Consistentes? -> Sí

=== Ejercicio 5: Intervalos de Fare y supervivencia por intervalo ===
Bordes de intervalos (linspace): [0.0, 102.4658, 204.9317, 307.3975, 409.8634, 512.3292]

Pasajeros por intervalo:
FareInterval
(-0.001

/tmp/ipython-input-1424421804.py:75: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survive_by_bin = df.groupby('FareInterval')['Survived'].mean()
